In [18]:
import urllib
f = urllib.urlretrieve ("http://www.analyticscertificate.com/SparkWorkshop/Files/spam.txt", "spam.txt")
spam_file = "./spam.txt"

f = urllib.urlretrieve ("http://www.analyticscertificate.com/SparkWorkshop/Files/ham.txt", "ham.txt")
ham_file = "./ham.txt"

os.getcwd()

'/gpfs/global_fs01/sym_shared/YPProdSpark/user/s82e-9eb5b98d683997-bd809a83de29/notebook/work'

In [8]:
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.feature import HashingTF

In [19]:
 # Load 2 types of emails from text files: spam and ham (non-spam).
    # Each line has text from one email.
spam = sc.textFile(spam_file)
ham = sc.textFile(ham_file)
spam.take(10)

[u'Dear sir, I am a Prince in a far kingdom you have not heard of.  I want to send you money via wire transfer so please ...',
 u'Get Viagra real cheap!  Send money right away to ...',
 u'Oh my gosh you can be really strong too with these drugs found in the rainforest. Get them cheap right now ...',
 u'YOUR COMPUTER HAS BEEN INFECTED!  YOU MUST RESET YOUR PASSWORD.  Reply to this email with your password and SSN ...',
 u'THIS IS NOT A SCAM!  Send money and get access to awesome stuff really cheap and never have to ...']

In [20]:
ham.take(10)

[u'Dear Spark Learner, Thanks so much for attending the Spark Summit 2014!  Check out videos of talks from the summit at ...',
 u'Hi Mom, Apologies for being late about emailing and forgetting to send you the package.  I hope you and bro have been ...',
 u'Wow, hey Fred, just heard about the Spark petabyte sort.  I think we need to take time to try it out immediately ...',
 u'Hi Spark user list, This is my first question to this list, so thanks in advance for your help!  I tried running ...',
 u"Thanks Tom for your email.  I need to refer you to Alice for this one.  I haven't yet figured out that part either ...",
 u'Good job yesterday!  I was attending your talk, and really enjoyed it.  I want to try out GraphX ...',
 u'Summit demo got whoops from audience!  Had to let you know. --Joe']

In [21]:
# Create a HashingTF instance to map email text to vectors of 100 features.
tf = HashingTF(numFeatures = 100)
# Each email is split into words, and each word is mapped to one feature.
spamFeatures = spam.map(lambda email: tf.transform(email.split(" ")))
hamFeatures = ham.map(lambda email: tf.transform(email.split(" ")))

In [22]:
# Create LabeledPoint datasets for positive (spam) and negative (ham) examples.
positiveExamples = spamFeatures.map(lambda features: LabeledPoint(1, features))
negativeExamples = hamFeatures.map(lambda features: LabeledPoint(0, features))
training_data = positiveExamples.union(negativeExamples)
training_data.cache() # Cache data since Logistic Regression is an iterative algorithm.

UnionRDD[41] at union at NativeMethodAccessorImpl.java:-2

In [23]:
# Run Logistic Regression using the SGD optimizer.
# regParam is model regularization, which can make models more robust.
model = LogisticRegressionWithSGD.train(training_data)

In [24]:
# Test on a positive example (spam) and a negative one (ham).
# First apply the same HashingTF feature transformation used on the training data.
posTestExample = tf.transform("O M G GET cheap stuff by sending money to ...".split(" "))
negTestExample = tf.transform("Hi Dad, I started studying Spark the other ...".split(" "))

In [25]:
# Now use the learned model to predict spam/ham for new emails.
print "Prediction for positive test example: %g" % model.predict(posTestExample)
print "Prediction for negative test example: %g" % model.predict(negTestExample)

Prediction for positive test example: 1
Prediction for negative test example: 0
